# Import Necessary Libraries

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
import collections
import os
import json
import pickle

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

# Load the Train Test Split

In [27]:
X_train = np.load(os.path.join("..", "IPD", "x_train.npy"))
X_test = np.load(os.path.join("..", "IPD", "x_test.npy"))
Y_train = np.load(os.path.join("..", "IPD", "y_train.npy"))
Y_test = np.load(os.path.join("..", "IPD", "y_test.npy"))
print("Training samples : {} \nTesting Samples : {}".format(X_train.shape, X_test.shape))
print("Training Labels : {} \nTesting Labels : {}".format(Y_train.shape, Y_test.shape))

Training samples : (332, 250, 12) 
Testing Samples : (83, 250, 12)
Training Labels : (332,) 
Testing Labels : (83,)


In [28]:
_ , num_timeframes, num_features = X_train.shape

# Reshape data for the model
-----------------
Samples are a 2D matrix which need to be reshaped to 1D to feed into the model

In [29]:
X_train = X_train.reshape((-1,num_timeframes*num_features))
X_test = X_test.reshape((-1,num_timeframes*num_features))
print("Training samples : {} \nTesting Samples : {}".format(X_train.shape, X_test.shape))
print("Training Labels : {} \nTesting Labels : {}".format(Y_train.shape, Y_test.shape))

Training samples : (332, 3000) 
Testing Samples : (83, 3000)
Training Labels : (332,) 
Testing Labels : (83,)


# Grid search over Random Forest model

In [38]:
parameters = {'bootstrap': [True, False], 
              'oob_score': [True, False],
              'max_depth': [6, 8, 10, 12, 14],
              'max_features': ['sqrt','log2',None],
              'n_estimators': [6,8,10,12,14]}

grid = GridSearchCV(RandomForestClassifier(), parameters, cv=3, verbose=2)
grid.fit(X_train, Y_train)

Fitting 3 folds for each of 300 candidates, totalling 900 fits
[CV] bootstrap=True, max_depth=6, max_features=sqrt, n_estimators=6, oob_score=True 
[CV]  bootstrap=True, max_depth=6, max_features=sqrt, n_estimators=6, oob_score=True, total=   0.0s
[CV] bootstrap=True, max_depth=6, max_features=sqrt, n_estimators=6, oob_score=True 
[CV]  bootstrap=True, max_depth=6, max_features=sqrt, n_estimators=6, oob_score=True, total=   0.0s
[CV] bootstrap=True, max_depth=6, max_features=sqrt, n_estimators=6, oob_score=True 
[CV]  bootstrap=True, max_depth=6, max_features=sqrt, n_estimators=6, oob_score=True, total=   0.0s
[CV] bootstrap=True, max_depth=6, max_features=sqrt, n_estimators=6, oob_score=False 
[CV]  bootstrap=True, max_depth=6, max_features=sqrt, n_estimators=6, oob_score=False, total=   0.0s
[CV] bootstrap=True, max_depth=6, max_features=sqrt, n_estimators=6, oob_score=False 
[CV]  bootstrap=True, max_depth=6, max_features=sqrt, n_estimators=6, oob_score=False, total=   0.0s
[CV] boo

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  bootstrap=True, max_depth=6, max_features=sqrt, n_estimators=8, oob_score=True, total=   0.0s
[CV] bootstrap=True, max_depth=6, max_features=sqrt, n_estimators=8, oob_score=True 
[CV]  bootstrap=True, max_depth=6, max_features=sqrt, n_estimators=8, oob_score=True, total=   0.0s
[CV] bootstrap=True, max_depth=6, max_features=sqrt, n_estimators=8, oob_score=False 
[CV]  bootstrap=True, max_depth=6, max_features=sqrt, n_estimators=8, oob_score=False, total=   0.0s
[CV] bootstrap=True, max_depth=6, max_features=sqrt, n_estimators=8, oob_score=False 
[CV]  bootstrap=True, max_depth=6, max_features=sqrt, n_estimators=8, oob_score=False, total=   0.0s
[CV] bootstrap=True, max_depth=6, max_features=sqrt, n_estimators=8, oob_score=False 
[CV]  bootstrap=True, max_depth=6, max_features=sqrt, n_estimators=8, oob_score=False, total=   0.0s
[CV] bootstrap=True, max_depth=6, max_features=sqrt, n_estimators=10, oob_score=True 
[CV]  bootstrap=True, max_depth=6, max_features=sqrt, n_estimators=1

[CV]  bootstrap=True, max_depth=6, max_features=log2, n_estimators=14, oob_score=True, total=   0.0s
[CV] bootstrap=True, max_depth=6, max_features=log2, n_estimators=14, oob_score=False 
[CV]  bootstrap=True, max_depth=6, max_features=log2, n_estimators=14, oob_score=False, total=   0.0s
[CV] bootstrap=True, max_depth=6, max_features=log2, n_estimators=14, oob_score=False 
[CV]  bootstrap=True, max_depth=6, max_features=log2, n_estimators=14, oob_score=False, total=   0.0s
[CV] bootstrap=True, max_depth=6, max_features=log2, n_estimators=14, oob_score=False 
[CV]  bootstrap=True, max_depth=6, max_features=log2, n_estimators=14, oob_score=False, total=   0.0s
[CV] bootstrap=True, max_depth=6, max_features=None, n_estimators=6, oob_score=True 
[CV]  bootstrap=True, max_depth=6, max_features=None, n_estimators=6, oob_score=True, total=   0.4s
[CV] bootstrap=True, max_depth=6, max_features=None, n_estimators=6, oob_score=True 
[CV]  bootstrap=True, max_depth=6, max_features=None, n_estima

[CV]  bootstrap=True, max_depth=8, max_features=sqrt, n_estimators=10, oob_score=True, total=   0.0s
[CV] bootstrap=True, max_depth=8, max_features=sqrt, n_estimators=10, oob_score=True 
[CV]  bootstrap=True, max_depth=8, max_features=sqrt, n_estimators=10, oob_score=True, total=   0.0s
[CV] bootstrap=True, max_depth=8, max_features=sqrt, n_estimators=10, oob_score=True 
[CV]  bootstrap=True, max_depth=8, max_features=sqrt, n_estimators=10, oob_score=True, total=   0.0s
[CV] bootstrap=True, max_depth=8, max_features=sqrt, n_estimators=10, oob_score=False 
[CV]  bootstrap=True, max_depth=8, max_features=sqrt, n_estimators=10, oob_score=False, total=   0.0s
[CV] bootstrap=True, max_depth=8, max_features=sqrt, n_estimators=10, oob_score=False 
[CV]  bootstrap=True, max_depth=8, max_features=sqrt, n_estimators=10, oob_score=False, total=   0.0s
[CV] bootstrap=True, max_depth=8, max_features=sqrt, n_estimators=10, oob_score=False 
[CV]  bootstrap=True, max_depth=8, max_features=sqrt, n_esti

...

[CV]  bootstrap=False, max_depth=12, max_features=None, n_estimators=8, oob_score=False, total=   1.5s
[CV] bootstrap=False, max_depth=12, max_features=None, n_estimators=8, oob_score=False 
[CV]  bootstrap=False, max_depth=12, max_features=None, n_estimators=8, oob_score=False, total=   1.6s
[CV] bootstrap=False, max_depth=12, max_features=None, n_estimators=10, oob_score=True 
[CV]  bootstrap=False, max_depth=12, max_features=None, n_estimators=10, oob_score=True, total=   0.0s
[CV] bootstrap=False, max_depth=12, max_features=None, n_estimators=10, oob_score=True 
[CV]  bootstrap=False, max_depth=12, max_features=None, n_estimators=10, oob_score=True, total=   0.0s
[CV] bootstrap=False, max_depth=12, max_features=None, n_estimators=10, oob_score=True 
[CV]  bootstrap=False, max_depth=12, max_features=None, n_estimators=10, oob_score=True, total=   0.0s
[CV] bootstrap=False, max_depth=12, max_features=None, n_estimators=10, oob_score=False 
[CV]  bootstrap=False, max_depth=12, max_fea

[CV]  bootstrap=False, max_depth=14, max_features=sqrt, n_estimators=14, oob_score=False, total=   0.1s
[CV] bootstrap=False, max_depth=14, max_features=sqrt, n_estimators=14, oob_score=False 
[CV]  bootstrap=False, max_depth=14, max_features=sqrt, n_estimators=14, oob_score=False, total=   0.1s
[CV] bootstrap=False, max_depth=14, max_features=log2, n_estimators=6, oob_score=True 
[CV]  bootstrap=False, max_depth=14, max_features=log2, n_estimators=6, oob_score=True, total=   0.0s
[CV] bootstrap=False, max_depth=14, max_features=log2, n_estimators=6, oob_score=True 
[CV]  bootstrap=False, max_depth=14, max_features=log2, n_estimators=6, oob_score=True, total=   0.0s
[CV] bootstrap=False, max_depth=14, max_features=log2, n_estimators=6, oob_score=True 
[CV]  bootstrap=False, max_depth=14, max_features=log2, n_estimators=6, oob_score=True, total=   0.0s
[CV] bootstrap=False, max_depth=14, max_features=log2, n_estimators=6, oob_score=False 
[CV]  bootstrap=False, max_depth=14, max_feature

[CV]  bootstrap=False, max_depth=14, max_features=None, n_estimators=8, oob_score=False, total=   1.6s
[CV] bootstrap=False, max_depth=14, max_features=None, n_estimators=10, oob_score=True 
[CV]  bootstrap=False, max_depth=14, max_features=None, n_estimators=10, oob_score=True, total=   0.0s
[CV] bootstrap=False, max_depth=14, max_features=None, n_estimators=10, oob_score=True 
[CV]  bootstrap=False, max_depth=14, max_features=None, n_estimators=10, oob_score=True, total=   0.0s
[CV] bootstrap=False, max_depth=14, max_features=None, n_estimators=10, oob_score=True 
[CV]  bootstrap=False, max_depth=14, max_features=None, n_estimators=10, oob_score=True, total=   0.0s
[CV] bootstrap=False, max_depth=14, max_features=None, n_estimators=10, oob_score=False 
[CV]  bootstrap=False, max_depth=14, max_features=None, n_estimators=10, oob_score=False, total=   1.8s
[CV] bootstrap=False, max_depth=14, max_features=None, n_estimators=10, oob_score=False 
[CV]  bootstrap=False, max_depth=14, max_f

[Parallel(n_jobs=1)]: Done 900 out of 900 | elapsed:  4.8min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [39]:
params = grid.best_params_
best_estimator = grid.best_estimator_
print(params)
print(best_estimator)

{'bootstrap': False, 'max_depth': 12, 'max_features': 'sqrt', 'n_estimators': 14, 'oob_score': False}
RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=12, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=14,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


In [40]:
clf = best_estimator
clf.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=12, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=14,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [41]:
Y_pred = clf.predict(X_test)
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.89      0.94      0.91        17
           1       0.83      0.83      0.83        12
           2       0.71      0.71      0.71        14
           3       1.00      0.94      0.97        16
           4       0.72      0.81      0.76        16
           5       0.17      0.12      0.14         8

    accuracy                           0.78        83
   macro avg       0.72      0.73      0.72        83
weighted avg       0.77      0.78      0.78        83

[[16  1  0  0  0  0]
 [ 0 10  1  0  1  0]
 [ 2  0 10  0  0  2]
 [ 0  0  0 15  0  1]
 [ 0  1  0  0 13  2]
 [ 0  0  3  0  4  1]]


# Saving model

In [42]:
pickle.dump(clf, open(os.path.join("..", "IPD", "RandomForest_model.pkl"), 'wb'))
print("Model Saved")

Model Saved
